In [ ]:
import numpy as np
from astropy.table import Table, join
from matplotlib import pyplot as plt

from bokeh.io import output_notebook, output_file, show, save
from bokeh.plotting import figure
from bokeh.palettes import Category20


## Combine summary tables from each fit run

In [ ]:
all_ia = Table.read('sncosmo_results/snia_all_bands/summary.csv')
blue_ia = Table.read('sncosmo_results/snia_ug/summary.csv')
red_ia = Table.read('sncosmo_results/snia_riz/summary.csv')
all_91bg = Table.read('sncosmo_results/91bg_all_bands/summary.csv')
blue_91bg = Table.read('sncosmo_results/91bg_ug/summary.csv')
red_91bg = Table.read('sncosmo_results/91bg_riz/summary.csv')

tables = (blue_ia, red_ia, blue_91bg, red_91bg)
table_names = ('blue_ia', 'red_ia', 'blue_91bg', 'red_91bg')
combined_table = None
for table, name in zip(tables, table_names):
    # Rename columns
    for column in table.colnames:
        if column in ('cid', 'class'):
            continue
            
        table.rename_column(column, column + '_' + name)
        
    # Join tables
    if combined_table is None:
        combined_table = table
        continue
        
    combined_table = join(combined_table, table, 
                          join_type='outer', 
                          keys=['cid', 'class'])


## Chi-squared Comparison

In [ ]:
# Get plot data
class_set = sorted(set(combined_table['class']))
x_vals = (combined_table['chi_blue_ia'] / combined_table['dof_blue_ia']
          - combined_table['chi_blue_91bg'] / combined_table['dof_blue_91bg'])

y_vals = (combined_table['chi_red_ia'] / combined_table['dof_red_ia'] 
          - combined_table['chi_red_91bg'] / combined_table['dof_red_91bg'])

# Plot data
fig = figure(plot_width=600, plot_height=400, 
             tools=['save', 'pan', 'box_zoom', 'reset', 'wheel_zoom'])
for classification, color in zip(class_set, Category20[20]):
    indices = combined_table['class'] == classification
    fig.circle(x_vals[indices], y_vals[indices], color=color, legend=classification)

# Move legend outside lot
fig.legend.click_policy="hide"
new_legend = fig.legend[0]
fig.legend[0].plot = None
fig.add_layout(new_legend, 'left')
fig.yaxis.axis_label = "Chi2 Ia - Chi2 91bg (riz)"
fig.xaxis.axis_label = "Chi2 Ia - Chi2 91bg (ug)"

output_notebook()
show(fig)


In [ ]:
output_file("chi_squared_comparison.html")
save(fig)
